In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.callback.progress import *
from local.vision.core import *

In [ ]:
#default_exp callback.mixup

# Mixup callback

> Callback to apply MixUp data augmentation to your training

In [ ]:
from local.utils.test import *

## MixupCallback -

In [ ]:
# export
def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

In [ ]:
# export
from torch.distributions.beta import Beta

class MixUp(Callback):
    _order = 90 #Runs after normalization and cuda
    def __init__(self, alpha = 0.4): self.distrib = Beta(tensor([alpha]), tensor([alpha]))

    def begin_fit(self): self.old_loss_func,self.learn.loss_func = self.loss_func,self.loss_func

    def begin_batch(self):
        if not self.training: return #Only mixup things during training
        lam = self.distrib.sample((self.yb.size(0),)).squeeze().to(self.xb.device)
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0][:,None,None,None]
        shuffle = torch.randperm(self.yb.size(0)).to(self.xb.device)
        xb1,self.yb1 = self.xb[shuffle],self.yb[shuffle]
        self.learn.xb = torch.lerp(xb1, self.xb, self.lam)

    def after_fit(self): self.run.loss_func = self.old_loss_func

    def loss_func(self, pred, yb):
        if not self.in_train: return self.old_loss_func(pred, yb)
        with NoneReduce(self.old_loss_func) as loss_func:
            loss1 = loss_func(pred, yb)
            loss2 = loss_func(pred, self.yb1)
        loss = torch.lerp(loss2, loss1, self.lam)
        return reduce_loss(loss, getattr(self.old_loss_func, 'reduction', 'mean'))

In [ ]:
#test on mnist
from local.vision.core import *

path = untar_data(URLs.MNIST_TINY)
items = get_image_files(path)
tds = TfmdDS(items, [PILImageBW.create, [parent_label, Categorize()]])
tdl = TfmdDL(tds, after_item=[ToTensor(), ByteToFloatTensor()], shuffle=True)

#TODO: make less ugly
mixup = MixUp()
mk_class('_FkLearner', xb=0, loss_func=CrossEntropyLossFlat(), training=True)
mixup.learn = _FkLearner()
mixup('begin_fit')
mixup.xb,mixup.yb = tdl.one_batch()
mixup('begin_batch')

In [ ]:
_,axs = plt.subplots(3,3, figsize=(9,9))
tdl.show_batch(b=(mixup.learn.xb,mixup.yb), ctxs=axs.flatten())

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)